In [3]:
using DataFrames, CSV
using JuMP, Gurobi

In [4]:
const GRB_ENV = Gurobi.Env();

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


In [19]:
data_o = CSV.read("train data/seed 10/train_o.csv", DataFrame);
data = CSV.read("train data/seed 10/train.csv", DataFrame);

In [14]:
names(data)

11-element Vector{String}:
 "Column1"
 "neigborhood_cat"
 "room_type"
 "minimum_nights"
 "number_of_reviews"
 "reviews_per_month"
 "calculated_host_listings_count"
 "availability_365"
 "number_of_reviews_ltm"
 "days_since_last_review"
 "price"

In [36]:
train_o = select(data_o, Not([:Column1,:number_of_reviews_ltm,:neigborhood_cat, :room_type]));
train = select(data, Not([:Column1,:number_of_reviews_ltm,:neigborhood_cat, :room_type]));

In [57]:
train_X[2,:]

,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count
,Float64,Float64,Float64,Float64
2,0.00177936,0.0324277,0.0224084,0.0


In [58]:
#Fitting stable regression on each dataset
function stable_regression_step1(X, y, k, λ; solver_output=0)
    n,p = size(X)
       
    # Build model
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", solver_output) 
    
    # Insert variables
    @variable(model, β[j=1:p])
    @variable(model, θ)
    @variable(model, u[1:n]>=0)
   
    #Insert constraints
    
    @constraint(model,[i=1:n], θ + u[i] >= y[i,1] - sum(X[i,j] * β[j] for j = 1:p))
    @constraint(model,[i=1:n],θ + u[i] >= -y[i,1] + sum(X[i,j] * β[j] for j = 1:p))
    
    #Objective
    @objective(model, Min, k*θ + sum(u[j] for j=1:n) + λ*sum(β.^2))
    
    # Optimize
    optimize!(model)
    
    # Return estimated betas
    return (value.(β))
end

stable_regression_step1 (generic function with 1 method)

In [59]:
train_X = select(train, Not([:price]));
train_y = select(train, :price);

In [60]:
size(train)[1] * 0.8

32443.2

In [61]:
beta = stable_regression_step1(train_X, train_y, 32443, 0.2)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


6-element Vector{Float64}:
  0.2353194220291636
 -0.16837230310209025
  1.1860551767031904
  0.24801979390147147
  0.10948737728802233
  0.12198711904345741

In [70]:
function stable_regression_step2(beta, X, y, k; solver_output = 0)
    n,p = size(X)
       
    # Build model
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", solver_output) 
    
    # Insert variables
    @variable(model, z[1:n]>=0)
    #@variable(model, t[1:n])
   
    #Insert constraints
    
    @constraint(model, sum(z) == k)
    @constraint(model,[i=1:n], z[i]<=1)
    #@constraint(model, [i = 1:n], t[i]<=y[i,1]-sum(X[i,j] * beta[j] for j = 1:p))
    #@constraint(model, [i = 1:n], t[i]<=-y[i,1]+sum(X[i,j] * beta[j] for j = 1:p))
    
    #Objective
    @objective(model, Max, sum(z[i] * abs(y[i,1] - sum(X[i,j] * beta[j] for j = 1:p)) for i = 1:n))
    
    # Optimize
    optimize!(model)
    
    # Return estimated betas
    return (value.(z))
end

stable_regression_step2 (generic function with 1 method)

In [71]:
z = stable_regression_step2(beta, train_X, train_y, 32443)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


40554-element Vector{Float64}:
 1.0
 1.0
 0.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 0.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.0
 0.0

In [73]:
train_stable = train[z.==1, :]

,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count
,Float64,Float64,Float64,Float64
1,0.00266904,0.0587204,0.0200701,0.0
2,0.00177936,0.0324277,0.0224084,0.0
3,0.0258007,0.000876424,0.0,0.00352113
4,0.00177936,0.00262927,0.00155885,0.0
5,0.00177936,0.022787,0.00643024,0.00352113
6,0.00088968,0.0,0.0175664,0.0
7,0.00088968,0.000876424,0.0192907,0.00704225
8,0.00088968,0.0,0.0200491,0.190141
9,0.0151246,0.0,0.0164275,0.0


In [74]:
train_stable_o = train_o[z.==1, :]

,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count
,Int64,Int64,Float64,Int64
1,3,2,0.03,4
2,1,0,0.778367,1
3,1,376,5.14,21
4,3,10,0.2,1
5,2,6,0.08,1
6,7,6,0.52,4
7,2,10,0.14,1
8,2,32,0.59,1
9,1,0,1.00948,1
